In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
class MyDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.transform = ToTensor()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        inputs = self.transform(sample['x'])
        label = torch.tensor(sample['y'])
        return inputs, label


In [3]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()

        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.out = nn.Sequential(nn.Linear(hidden_size, 64),nn.tanh(),nn.Linear(64, output_size),nn.Softmax(dim=output_size))

    def forward(self, input, hidden):
        batch_size = input.size(0)
        output, hidden = self.rnn(input, hidden)
        output = self.out(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size)

In [4]:
# Hyperparameter
input_size = 10
hidden_size = 20
output_size = 2
batch_size = 32
num_epochs = 23

In [5]:
# Create an instance of the RNN classifier
model = RNNClassifier(input_size, hidden_size, output_size)

# Load data from CSV file
csv_file = 'train.csv'
dataset = MyDataset(csv_file)

# Create a dataloader to handle batching and shuffling
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

csv_file = 'data.csv'
dataset = MyDataset(csv_file)

# Create a dataloader to handle batching and shuffling
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

FileNotFoundError: ignored

In [ ]:
# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_dataloader:
      hidden = model.init_hidden()
      # Zero the gradients
      optimizer.zero_grad()

      # Forward pass
      output = model(input, hidden)

      # Compute loss
      loss = criterion(output, labels)

      # Backward pass and optimization
      loss.backward()
      optimizer.step()

      # Print loss for monitoring
      print(f"Epoch: {epoch+1}, Batch Loss: {loss.item()}")

In [ ]:
# Set the model to evaluation mode
model.eval()

# Iterate over the test dataset
predictions = []
ground_truth = []
for inputs, labels in test_dataloader:
    # Forward pass
    outputs = model(inputs)

    # Get the predicted labels
    _, predicted = torch.max(outputs, dim=1)
    predictions += predicted.tolist()
    ground_truth += labels.tolist()

# Convert lists to numpy arrays
predictions = np.array(predictions)
ground_truth = np.array(ground_truth)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(ground_truth, predictions)
print(f"Accuracy: {accuracy}")